In [15]:
import polars as pl
import geopandas as gpd
from utils.loader_local import LoaderLocal


In [41]:
# Read data
def read_etat_des_ascenceurs():
    df = pl.read_csv('/home/onyxia/work/hackathon_mobilites_2025/data/raw/etat-des-ascenseurs.csv', 
    separator=";",
    truncate_ragged_lines=True)
    return df

In [53]:
df_ascensceurs = read_etat_des_ascenceurs()
print('df_ascensceurs number of rows : %i' %df_ascensceurs.shape[0])

df_asc_counts = (
    df_ascensceurs
        .group_by("zdcid")
        .agg(
            pl.col("liftid").n_unique().alias("n_lifts")
        )
)
print('df_ascensceurs_count_by_stations number of rows : %i' %df_asc_counts.shape[0])

df_ascensceurs number of rows : 944
df_ascensceurs_count_by_stations number of rows : 313


In [57]:
ref_table_finale_path = "/home/onyxia/work/hackathon_mobilites_2025/data/enrich/final_table.gpq"

stations = LoaderLocal.loader_geoparquet(ref_table_finale_path)
print('stations number of rows : %i' %stations.shape[0])

stations number of rows : 590


In [ ]:
#Attention : format particulier de l'id dans stations
stations_pl.filter(pl.col('id_ref_zdc')=='71485.0')

geo_point_2d,id_ref_zdc,nom_zda,station_clean,res_com,mode,exploitant,ligne,station,facilite_acces_code,facilite_acces,nombre_facilite_acces_station,id_zdc,total_validation,total_validation_amethyste,pct_amethyste
str,str,str,str,str,str,str,str,str,str,str,i64,str,f64,f64,f64
"""48.88813847761218, 2.249792769…","""71485.0""","""Esplanade de La Défense""","""esplanadedeladéfense""","""METRO 1""","""METRO""","""RATP""",null,"""Esplanade de La Défense""","""green""","""très facile d'accès (ascenseur…",1,"""71485.0""",1.3710823e8,1.958689e7,14.2857


In [59]:
#Cast variables for join
stations_geometry = stations.geometry.copy()
stations_pl = pl.from_pandas(stations.drop(columns="geometry"))

df_asc_counts = df_asc_counts.with_columns(
    pl.col("zdcid").cast(pl.Utf8)
)

print('df_asc_counts')
print(df_asc_counts.select('zdcid').head())

stations_pl = stations_pl.with_columns(
    pl.col("id_ref_zdc")
    .cast(pl.Float64, strict=False)   # 1) cast to float
    .cast(pl.Int64,   strict=False)   # 2) cast to int
    .cast(pl.Utf8,    strict=False)   # 3) cast to string
)
print('stations')
print(stations_pl.select('id_ref_zdc').head())

df_asc_counts
shape: (5, 1)
┌───────┐
│ zdcid │
│ ---   │
│ str   │
╞═══════╡
│ 67625 │
│ 70133 │
│ 72287 │
│ 71828 │
│ 68129 │
└───────┘
stations
shape: (5, 1)
┌────────────┐
│ id_ref_zdc │
│ ---        │
│ str        │
╞════════════╡
│ 71432      │
│ 73669      │
│ 72491      │
│ 71030      │
│ 71632      │
└────────────┘


In [60]:
joined_pl = stations_pl.join(
    df_asc_counts,
    left_on="id_ref_zdc",
    right_on="zdcid",
    how="left"
)
joined_gdf = gpd.GeoDataFrame(joined_pl.to_pandas(), geometry=stations_geometry)

In [62]:
joined_gdf

,geo_point_2d,id_ref_zdc,nom_zda,station_clean,res_com,mode,exploitant,ligne,station,facilite_acces_code,facilite_acces,nombre_facilite_acces_station,id_zdc,total_validation,total_validation_amethyste,pct_amethyste,n_lifts,geometry
0,"48.884594208027806, 2.3379497756920697",71432,Abbesses,abbesses,METRO 12,METRO,RATP,None,Abbesses,yellow,équipée d'au moins un escalator ou ascenseur s...,1,71432.0,135006480.0,19286640.0,14.2857,2.0,POINT (48.88459 2.33795)
1,"48.872097029931695, 2.40862005517699",73669,Adrienne Bolland,adriennebolland,TRAM 3b,TRAMWAY,RATP,None,Adrienne Bolland,green,très facile d'accès (ascenseur ou plain-pied),1,None,NaN,NaN,NaN,NaN,POINT (48.8721 2.40862)
2,"48.90808243782026, 2.377741481106109",72491,Aimé Césaire,aimécésaire,METRO 12,METRO,RATP,None,Aimé Césaire,black,"facile d'accès, équipée d'escalator ou ascense...",1,72491.0,138602792.0,19862534.0,14.3305,3.0,POINT (48.90808 2.37774)
3,"48.82818847631351, 2.3271646496078247",71030,Alésia,alésia,METRO 4,METRO,RATP,None,Alésia,yellow,équipée d'au moins un escalator ou ascenseur s...,1,71030.0,136235540.0,19462220.0,14.2857,NaN,POINT (48.82819 2.32716)
4,"48.843700739992194, 2.410109555597318",71632,Alexandra David-Néel,alexandradavidnéel,TRAM 3a,TRAMWAY,RATP,None,Alexandra David-Néel,green,très facile d'accès (ascenseur ou plain-pied),1,None,NaN,NaN,NaN,NaN,POINT (48.8437 2.41011)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,"48.84734036417028, 2.432434660260661",71651,Vincennes,vincennes,RER A,RER,RATP,None,Vincennes,green,très facile d'accès (ascenseur ou plain-pied),1,71651.0,137354967.0,19632374.0,14.2932,2.0,POINT (48.84734 2.43243)
586,"48.80020939834459, 2.40271281718153",70313,Vitry-sur-Seine,vitrysurseine,RER C,RER,SNCF,None,Vitry-sur-Seine,green,très facile d'accès (ascenseur ou plain-pied),1,70313.0,133313448.0,19265762.0,14.4515,2.0,POINT (48.80021 2.40271)
587,"48.84150552236008, 2.3079486779742893",71113,Volontaires,volontaires,METRO 12,METRO,RATP,None,Volontaires,white,comportant uniquement des escaliers et peu pro...,1,71113.0,135683604.0,19484962.0,14.3606,NaN,POINT (48.84151 2.30795)
588,"48.85787849486676, 2.3800206442479825",71750,Voltaire,voltaire,METRO 9,METRO,RATP,None,Voltaire,yellow,équipée d'au moins un escalator ou ascenseur s...,1,71750.0,135894500.0,19372500.0,14.2555,NaN,POINT (48.85788 2.38002)


In [ ]:
match_col = "n_lifts"   # typically the join key is good

matched_count = joined_pl.filter(
    pl.col(match_col).is_not_null()
).count()

not_matched_count = joined_pl.filter(
    pl.col(match_col).is_null()
).count()

total = joined_pl.count()

matched_count, not_matched_count, total

(shape: (1, 29)
 ┌────────────┬────────────┬─────────┬────────────┬───┬───────────┬─────────┬──────────┬────────────┐
 │ geo_point_ ┆ id_ref_zdc ┆ nom_zda ┆ station_cl ┆ … ┆ liftstate ┆ zdcname ┆ liftmode ┆ centroidzd │
 │ 2d         ┆ ---        ┆ ---     ┆ ean        ┆   ┆ ---       ┆ ---     ┆ ---      ┆ c          │
 │ ---        ┆ u32        ┆ u32     ┆ ---        ┆   ┆ u32       ┆ u32     ┆ u32      ┆ ---        │
 │ u32        ┆            ┆         ┆ u32        ┆   ┆           ┆         ┆          ┆ u32        │
 ╞════════════╪════════════╪═════════╪════════════╪═══╪═══════════╪═════════╪══════════╪════════════╡
 │ 678        ┆ 678        ┆ 678     ┆ 678        ┆ … ┆ 675       ┆ 678     ┆ 678      ┆ 678        │
 └────────────┴────────────┴─────────┴────────────┴───┴───────────┴─────────┴──────────┴────────────┘,
 shape: (1, 29)
 ┌────────────┬────────────┬─────────┬────────────┬───┬───────────┬─────────┬──────────┬────────────┐
 │ geo_point_ ┆ id_ref_zdc ┆ nom_zda ┆ station_cl